In [1]:
from statistics import mean, stdev
from transformers import BertTokenizer
from tqdm.notebook import tqdm
from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
# from datasets.reader import read_conll
from datasets.conll import ConllBertDataset

# Load data

In [2]:
manga_path = '../../data/NER/processed/comments/augmented_10/'

In [3]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-conversational')
train_ds = ConllBertDataset.from_file(manga_path + 'train.txt', tokenizer)
test_ds = ConllBertDataset.from_file(manga_path + 'test.txt', tokenizer)

  0%|          | 0/2014 [00:00<?, ?it/s]

  0%|          | 0/503 [00:00<?, ?it/s]

In [4]:
train_ds[-1:]

([[101, 2063, 801, 28293, 845, 1702, 10029, 292, 292, 102]],
 [['O', 'O', 'O', 'PER', 'PER', 'PER', 'O', 'O', 'O', 'O']])

In [5]:
type(train_ds[0])

tuple

# Create model

In [7]:
import torch
import torch.nn as nn

from models import BertLstm
from sklearn.metrics import roc_auc_score

In [8]:
BertLstm

models.naive.BertLstm

In [9]:
acc = lambda preds, labels: ((preds.argmax(dim=1) == labels).sum() / labels.size(0)).item()
sigmoid = nn.Sigmoid()

def auc(preds, labels):
    assert all(map(lambda preds: preds.size(1) == 2, preds))
    to_probs = lambda preds: sigmoid(preds)[:, 0].cpu().detach().numpy()
    to_labels = lambda labels: (1 - labels).cpu().detach().numpy()
    
    preds = list(map(to_probs, preds))
    labels = list(map(to_labels, labels))
    
    preds = [item for subl in preds for item in subl]
    labels = [item for subl in labels for item in subl]
    
    auc = roc_auc_score(labels, preds)
    return auc

In [70]:
import numpy as np
import sklearn
from sklearn.metrics import precision_recall_curve
T=0.5
def f_score(preds, labels):
    preds2 = []
    assert all(map(lambda preds: preds.size(1) == 2, preds))
    to_probs = lambda preds: sigmoid(preds)[:, 0].cpu().detach().numpy()
    to_labels = lambda labels: (1 - labels).cpu().detach().numpy()
    
    preds = list(map(to_probs, preds))
    labels = list(map(to_labels, labels))
    
    preds = [item for subl in preds for item in subl]
    labels = [item for subl in labels for item in subl]
    precision, recall, thresholds = precision_recall_curve(labels, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    ix = np.argmax(fscore)
    print(thresholds[ix])
    return fscore[ix]

# Train

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
labels_n = max(train_ds.labels_n, test_ds.labels_n)

simple_model = BertLstm(labels_n).to(device)
opt = torch.optim.Adam(simple_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

writer = SummaryWriter()#'runs/16_epoches_30_augs_uncased')

c:\users\май\appdata\local\programs\python\python36\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [72]:
loss_interval = 50

for e in range(10):
    print('Epoch', e+1) 
    losses = []
    probs_epoch = []
    labels_epoch = []
    opt.zero_grad()
    for i in tqdm(range(1, len(train_ds))):
        if i % loss_interval == 0:
            opt.step()
            opt.zero_grad()
        words, labels = train_ds[i]
        words = torch.LongTensor(words).unsqueeze(0).to(device)
        labels = torch.LongTensor(list(map(lambda label: train_ds.ne2ix[label], labels))).to(device)

        preds = simple_model(words)
        loss = criterion(preds, labels)
        loss.backward()
        step = e*len(train_ds)+i
        writer.add_scalar('Loss/Train', loss.item(), step)
        writer.add_scalar('Accuracy/Train', acc(preds, labels), step)
        probs_epoch.append(preds)
        labels_epoch.append(labels)
    print(f_score(probs_epoch, labels_epoch))
    writer.add_scalar('AUC_epoch/Train', auc(probs_epoch, labels_epoch), e) 
            
    losses = []
    probs_epoch = []
    labels_epoch = []
    with torch.no_grad():
        for i in tqdm(range(1, len(test_ds))):
            words, labels = train_ds[i]
            words = torch.LongTensor(words).unsqueeze(0).to(device)
            labels = torch.LongTensor(list(map(lambda label: test_ds.ne2ix[label], labels))).to(device)
            preds = simple_model(words)
            loss = criterion(preds, labels)
            step = e*len(test_ds)+i
            writer.add_ueuscalar('Loss/Test', loss.item(), step)
            writer.add_scalar('Accuracy/Test', acc(preds, labels), step)
            probs_epoch.append(preds)
            labels_epoch.append(labels)
    print(f_score(probs_epoch, labels_epoch))
    writer.add_scalar('AUC_epoch/Test', auc(probs_epoch, labels_epoch), e) 
    

Epoch 1


  0%|          | 0/2013 [00:00<?, ?it/s]

0.5798725
0.8515026517383618


  0%|          | 0/502 [00:00<?, ?it/s]

0.5720629
0.8476070528967256
Epoch 2


...

0.63887805
0.8679479231246124
Epoch 10


  0%|          | 0/2013 [00:00<?, ?it/s]

0.5154614
0.8799538838449344


  0%|          | 0/502 [00:00<?, ?it/s]

0.89143866
0.8740740740740741


In [64]:
simple_model.save('./weights/model.pt')